In [18]:
from pprint import pprint
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 144
import itertools as it
from functools import reduce
import operator as op
import numpy as np

In [19]:
import sys
(sys.version, pd.__version__, sns.__version__, plt.matplotlib.__version__, np.__version__)

('3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]',
 '1.2.4',
 '0.11.1',
 '3.3.4',
 '1.20.1')

In [20]:
#with open('dat/rates/SectorEcondAsicTestStatsPD_2.txt') as f:
    #print(f.read())

In [21]:
with open('dat/rates/Version10_hits.txt') as f:
    print(f.read())

  1   1   2   0   d 160   0   d 157  85.968000   1.020840   9.612600   1.000021  32.031400   1.023702  73.382000
  1   1   3   0   F 432   0   F 432 180.675400   1.000000  15.449800   1.000000  50.619000   1.000000 141.395600
  1   1   4   0   F 432   0   F 432  96.043600   1.000000   6.269400   1.000000  17.898400   1.000000  87.163400
  1   1   5   0   F 432   0   F 432  54.437400   1.000000   3.224200   1.000000   8.518200   1.000000  62.891200
  1   1   6   1   F 192   1   F 192  33.267600   1.000000   7.478600   1.000000   9.713000   1.000000  39.998000
  1   1   7   1   F 192   1   F 192  20.800800   1.000000   4.455400   1.000000   5.898200   1.000000  31.450400
  1   1   8   1   F 192   1   F 192  12.228400   1.000000   3.084600   1.000000   2.062400   1.000000  24.793600
  1   1   9   1   b 164   1   F 192   5.835600   0.856501   1.510000   0.936291   1.090200   0.955971  15.669400
  1   2   0   0   g 307   0   g 294 160.228800   1.044303  16.802200   1.050279  56.799400   1.0

In [22]:
rates_col_description = {
    'Layer': 'Module Layer',
    'U': 'Module U',
    'V': 'Module V',
    'Type': 'Module type',
    'Shape': 'Module shape',
    'cells': 'cells',
    "cmssw_type" : "cmssw_type",
    "cmssw_shape" : "cmssw_shape",
    "cmssw_cells" : "cmssw_cells" ,
    "nHitsTotal" : "nHitsTotal" ,
    "corrFactorTotal" : "corrFactorTotal", 
    "nHistsToA" : "nHistsToA" ,
    "corrFactorToA" : "corrFactorToA" ,
    "nHitsBXM1" : "nHitsBXM1",
    "corrFactorBXM1" : "corrFactorBXM1",
    'EvSize': 'Average event size [32b words]'
}

#Extract the column labels for the data frame (freeze it in a tuple)
rates_col_labels=tuple(rates_col_description.keys())

#Make the main description dict reversible for easy .rename() of columns back and forth
rates_col_description.update(
    {long: short for (short, long) in rates_col_description.items()}
    )

pprint(rates_col_labels)

('Layer',
 'U',
 'V',
 'Type',
 'Shape',
 'cells',
 'cmssw_type',
 'cmssw_shape',
 'cmssw_cells',
 'nHitsTotal',
 'corrFactorTotal',
 'nHistsToA',
 'corrFactorToA',
 'nHitsBXM1',
 'corrFactorBXM1',
 'EvSize')


In [23]:
rates_col_description

{'Layer': 'Module Layer',
 'U': 'Module U',
 'V': 'Module V',
 'Type': 'Module type',
 'Shape': 'Module shape',
 'cells': 'cells',
 'cmssw_type': 'cmssw_type',
 'cmssw_shape': 'cmssw_shape',
 'cmssw_cells': 'cmssw_cells',
 'nHitsTotal': 'nHitsTotal',
 'corrFactorTotal': 'corrFactorTotal',
 'nHistsToA': 'nHistsToA',
 'corrFactorToA': 'corrFactorToA',
 'nHitsBXM1': 'nHitsBXM1',
 'corrFactorBXM1': 'corrFactorBXM1',
 'EvSize': 'Average event size [32b words]',
 'Module Layer': 'Layer',
 'Module U': 'U',
 'Module V': 'V',
 'Module type': 'Type',
 'Module shape': 'Shape',
 'Average event size [32b words]': 'EvSize'}

In [24]:
module_type = {
    0: 'HD',
    1: 'LD',
}
module_type.update(
    { i: f'Sci{i-2}' for i in range(2,12) }
)
pprint(module_type)

{0: 'HD',
 1: 'LD',
 2: 'Sci0',
 3: 'Sci1',
 4: 'Sci2',
 5: 'Sci3',
 6: 'Sci4',
 7: 'Sci5',
 8: 'Sci6',
 9: 'Sci7',
 10: 'Sci8',
 11: 'Sci9'}


In [25]:
#def make_LUV(df):
     #return 256*df['Layer'] + 16*df['U'] + df['V']

#def make_LUV_index(df):
    #df['LUV'] = make_LUV(df)
    #df.set_index('LUV', inplace=True, verify_integrity=True)

In [26]:
rates = pd.read_table(
    'dat/rates/Version10_hits.txt',
    sep='\s+',
    header=None,
    names=rates_col_labels,
    converters={
        'type': lambda x: module_type[int(x)],
        },
    dtype={
        'l': np.uint8,
        'Elinks': np.int16,
        'u': np.int8,
        'v': np.int8,
        }  
    )

In [27]:
rates.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Layer            4500 non-null   int64  
 1   U                4500 non-null   int64  
 2   V                4500 non-null   int64  
 3   Type             4500 non-null   int64  
 4   Shape            4500 non-null   object 
 5   cells            4500 non-null   int64  
 6   cmssw_type       4500 non-null   int64  
 7   cmssw_shape      4500 non-null   object 
 8   cmssw_cells      4500 non-null   int64  
 9   nHitsTotal       4500 non-null   float64
 10  corrFactorTotal  4500 non-null   float64
 11  nHistsToA        4500 non-null   float64
 12  corrFactorToA    4500 non-null   float64
 13  nHitsBXM1        4500 non-null   float64
 14  corrFactorBXM1   4500 non-null   float64
 15  EvSize           4500 non-null   float64
dtypes: float64(7), int64(7), object(2)
memory usage: 1002.1 KB


In [28]:
rates

,Layer,U,V,Type,Shape,cells,cmssw_type,cmssw_shape,cmssw_cells,nHitsTotal,corrFactorTotal,nHistsToA,corrFactorToA,nHitsBXM1,corrFactorBXM1,EvSize
0,1,1,2,0,d,160,0,d,157,85.9680,1.02084,9.6126,1.000021,32.0314,1.023702,73.3820
1,1,1,3,0,F,432,0,F,432,180.6754,1.00000,15.4498,1.000000,50.6190,1.000000,141.3956
2,1,1,4,0,F,432,0,F,432,96.0436,1.00000,6.2694,1.000000,17.8984,1.000000,87.1634
3,1,1,5,0,F,432,0,F,432,54.4374,1.00000,3.2242,1.000000,8.5182,1.000000,62.8912
4,1,1,6,1,F,192,1,F,192,33.2676,1.00000,7.4786,1.000000,9.7130,1.000000,39.9980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,47,15,7,11,f,312,2,F,304,0.4984,1.00000,0.1976,1.000000,0.0000,0.000000,15.9518
4496,47,15,8,11,F,320,2,F,312,0.6030,1.00000,0.2470,1.000000,0.0000,0.000000,16.1394
4497,47,15,9,11,F,320,2,F,312,0.5786,1.00000,0.2312,1.000000,0.0000,0.000000,16.0978
4498,47,15,10,11,f,312,2,F,304,0.5316,1.00000,0.2214,1.000000,0.0000,0.000000,16.0136


In [29]:
# delete a single row by index value 0
#rates = rates.drop(labels=0, axis=0,inplace=True)
#rates



In [30]:
rates.to_csv('module_rates_used for version10.csv') 

In [31]:
import json

In [32]:
json_file_path = "FeMappingV10.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [33]:
contents

[{'layer': 1,
  'MB': 0,
  'type': 0,
  'lpGBTs': [0, 1],
  'modules': [{'u': 2, 'v': 0, 'elinks': [4, 0], 'shape': 'g'},
   {'u': 3, 'v': 0, 'elinks': [3, 0], 'shape': 'F'},
   {'u': 4, 'v': 0, 'elinks': [0, 2], 'shape': 'F'}]},
 {'layer': 1,
  'MB': 1,
  'type': 0,
  'lpGBTs': [2],
  'modules': [{'u': 4, 'v': 1, 'elinks': [3], 'shape': 'F'},
   {'u': 5, 'v': 1, 'elinks': [2], 'shape': 'F'},
   {'u': 5, 'v': 2, 'elinks': [2], 'shape': 'F'}]},
 {'layer': 1,
  'MB': 2,
  'type': 0,
  'lpGBTs': [3, 4],
  'modules': [{'u': 2, 'v': 1, 'elinks': [0, 2], 'shape': 'd'},
   {'u': 3, 'v': 2, 'elinks': [4, 0], 'shape': 'F'},
   {'u': 4, 'v': 3, 'elinks': [3, 0], 'shape': 'F'},
   {'u': 5, 'v': 4, 'elinks': [0, 2], 'shape': 'F'}]},
 {'layer': 1,
  'MB': 3,
  'type': 0,
  'lpGBTs': [5, 6],
  'modules': [{'u': 3, 'v': 1, 'elinks': [4, 0], 'shape': 'F'},
   {'u': 4, 'v': 2, 'elinks': [3, 0], 'shape': 'F'},
   {'u': 5, 'v': 3, 'elinks': [0, 2], 'shape': 'F'}]},
 {'layer': 1,
  'MB': 4,
  'type': 1,
 

In [17]:
rates.loc[rates["Type"] == "HD", "Type"] = 0
rates.loc[rates["Type"] == "LD", "Type"] = 1
for i in range (2,12):
    rates.loc[rates["Type"] == f'Sci{i-2}', "Type"] = int(i)
rates

,Layer,U,V,Type,Shape,cells,cmssw_type,cmssw_shape,cmssw_cells,nHitsTotal,corrFactorTotal,nHistsToA,corrFactorToA,nHitsBXM1,corrFactorBXM1,EvSize
0,1,1,2,0,d,160,0,d,157,85.9680,1.02084,9.6126,1.000021,32.0314,1.023702,73.3820
1,1,1,3,0,F,432,0,F,432,180.6754,1.00000,15.4498,1.000000,50.6190,1.000000,141.3956
2,1,1,4,0,F,432,0,F,432,96.0436,1.00000,6.2694,1.000000,17.8984,1.000000,87.1634
3,1,1,5,0,F,432,0,F,432,54.4374,1.00000,3.2242,1.000000,8.5182,1.000000,62.8912
4,1,1,6,1,F,192,1,F,192,33.2676,1.00000,7.4786,1.000000,9.7130,1.000000,39.9980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,47,15,7,11,f,312,2,F,304,0.4984,1.00000,0.1976,1.000000,0.0000,0.000000,15.9518
4496,47,15,8,11,F,320,2,F,312,0.6030,1.00000,0.2470,1.000000,0.0000,0.000000,16.1394
4497,47,15,9,11,F,320,2,F,312,0.5786,1.00000,0.2312,1.000000,0.0000,0.000000,16.0978
4498,47,15,10,11,f,312,2,F,304,0.5316,1.00000,0.2214,1.000000,0.0000,0.000000,16.0136


In [16]:
''''def get_Event_Size_lpGBTs_(i):
    sum_rate=0
    if len(contents[i]["lpGBTs"])==1:
        for k in range(len(contents[i]["modules"])):
            rate=rates.loc[(rates["Layer"]==int(contents[i]["layer"]))& (rates["Type"]==int(contents[i]["type"]))&
                    (rates["Shape"]==contents[i]["modules"][k]["shape"])&(rates["U"]==int(contents[i]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[i]["modules"][k]["v"]))]["EvSize"].item()
            sum_rate+=rate
            
    else:
        for k in range(len(contents[i]["modules"])):
            rate=rates.loc[(rates["Layer"]==int(contents[i]["layer"]))& (rates["Type"]==int(contents[i]["type"]))&
                    (rates["Shape"]==contents[i]["modules"][k]["shape"])&(rates["U"]==int(contents[i]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[i]["modules"][k]["v"]))]["EvSize"].item()
            sum_rate+=rate
        
    return sum_rate

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-16-7635a58a673f>, line 17)

In [18]:
#There is no shape and elink info
def get_Event_Size_lpGBTs_(i):
    sum_rate=0
    
    if len(contents[i]["lpGBTs"])==1:
        for k in range(len(contents[i]["modules"])):
            rate=rates.loc[(rates["Layer"]==int(contents[i]["layer"]))& (rates["Type"]==int(contents[i]["type"]))&
                    (rates["U"]==int(contents[i]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[i]["modules"][k]["v"]))]["EvSize"].item()
            sum_rate+=rate
            
    else:
        for k in range(len(contents[i]["modules"])):
            rate=rates.loc[(rates["Layer"]==int(contents[i]["layer"]))& (rates["Type"]==int(contents[i]["type"]))&
                    (rates["U"]==int(contents[i]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[i]["modules"][k]["v"]))]["EvSize"].item()
            sum_rate+=rate
    #count+=1
        
    return sum_rate
   

In [19]:
def update_contents():
    for i in range(len(contents)):
        event_size=get_Event_Size_lpGBTs_(i)
        #print(event_size)
        contents[i].pop("modules")
        contents[i]["eventSize"]=event_size
        lpgbt_id=contents[i]["lpGBTs"][0]
        contents[i]["lpgbt_id"]=lpgbt_id
    
        if len(contents[i]["lpGBTs"])==2:
            contents[i]["doublelpGBT"]=True
        else:
            contents[i]["doublelpGBT"]=False
            
    return contents


In [20]:
update_contents()

[{'layer': 1,
  'MB': 0,
  'type': 0,
  'lpGBTs': [0, 1],
  'eventSize': 323.57439999999997,
  'lpgbt_id': 0,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 1,
  'type': 0,
  'lpGBTs': [2],
  'eventSize': 217.00220000000002,
  'lpgbt_id': 2,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 2,
  'type': 0,
  'lpGBTs': [3, 4],
  'eventSize': 364.53499999999997,
  'lpgbt_id': 3,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 3,
  'type': 0,
  'lpGBTs': [5, 6],
  'eventSize': 300.3104,
  'lpgbt_id': 5,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 4,
  'type': 1,
  'lpGBTs': [7],
  'eventSize': 111.7748,
  'lpgbt_id': 7,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 5,
  'type': 1,
  'lpGBTs': [8],
  'eventSize': 126.94200000000001,
  'lpgbt_id': 8,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 6,
  'type': 1,
  'lpGBTs': [9],
  'eventSize': 138.88180000000003,
  'lpgbt_id': 9,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 7,
  'type': 1,
  'lpGBTs': [10],
  'eventSize': 127.13000000000001,

In [31]:
''''for k in range(len(contents[769]["modules"])):
    
    rate=rates.loc[(rates["Layer"]==int(contents[769]["layer"]))& (rates["Type"]==int(contents[769]["type"]))&
                    (rates["U"]==int(contents[769]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[769]["modules"][k]["v"]))]#["EvSize"].item()
    print(rate)

Empty DataFrame
Columns: [Layer, U, V, Type, Shape, ADCHits, TOXHits, EvSize, EvSizeXCheck, Elinks]
Index: []
      Layer  U  V Type Shape  ADCHits   TOXHits  EvSize  EvSizeXCheck  Elinks
3054     31  6  4    1     F  4.42063  0.504683  19.917            20       1
      Layer  U  V Type Shape  ADCHits   TOXHits   EvSize  EvSizeXCheck  Elinks
3055     31  7  5    1     F  1.71947  0.454168  17.1726            17       1
      Layer  U  V Type Shape   ADCHits  TOXHits  EvSize  EvSizeXCheck  Elinks
3056     31  8  6    1     F  0.872969  0.26873  16.141            16       1
      Layer  U  V Type Shape   ADCHits   TOXHits   EvSize  EvSizeXCheck  \
3057     31  9  7    1     F  0.519581  0.157148  15.6764            16   

      Elinks  
3057       1  
      Layer   U  V Type Shape   ADCHits   TOXHits   EvSize  EvSizeXCheck  \
3058     31  10  8    1     F  0.309312  0.103246  15.4124            15   

      Elinks  
3058       1  


In [21]:
contents

[{'layer': 1,
  'MB': 0,
  'type': 0,
  'lpGBTs': [0, 1],
  'eventSize': 323.57439999999997,
  'lpgbt_id': 0,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 1,
  'type': 0,
  'lpGBTs': [2],
  'eventSize': 217.00220000000002,
  'lpgbt_id': 2,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 2,
  'type': 0,
  'lpGBTs': [3, 4],
  'eventSize': 364.53499999999997,
  'lpgbt_id': 3,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 3,
  'type': 0,
  'lpGBTs': [5, 6],
  'eventSize': 300.3104,
  'lpgbt_id': 5,
  'doublelpGBT': True},
 {'layer': 1,
  'MB': 4,
  'type': 1,
  'lpGBTs': [7],
  'eventSize': 111.7748,
  'lpgbt_id': 7,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 5,
  'type': 1,
  'lpGBTs': [8],
  'eventSize': 126.94200000000001,
  'lpgbt_id': 8,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 6,
  'type': 1,
  'lpGBTs': [9],
  'eventSize': 138.88180000000003,
  'lpgbt_id': 9,
  'doublelpGBT': False},
 {'layer': 1,
  'MB': 7,
  'type': 1,
  'lpGBTs': [10],
  'eventSize': 127.13000000000001,

In [22]:
input_file=pd.DataFrame(contents)
input_file

,layer,MB,type,lpGBTs,eventSize,lpgbt_id,doublelpGBT
0,1,0,0,"[0, 1]",323.5744,0,True
1,1,1,0,[2],217.0022,2,False
2,1,2,0,"[3, 4]",364.5350,3,True
3,1,3,0,"[5, 6]",300.3104,5,True
4,1,4,1,[7],111.7748,7,False
...,...,...,...,...,...,...,...
1147,47,0,11,[2163],15.9518,2163,False
1148,47,0,11,[2164],16.1394,2164,False
1149,47,0,11,[2165],16.0978,2165,False
1150,47,0,11,[2166],16.0136,2166,False


In [23]:
input_file.to_csv('input_file_version10.csv') 

In [34]:
pd.set_option('display.max_rows', None)
rates

,Layer,U,V,Type,Shape,ADCHits,TOXHits,EvSize,EvSizeXCheck,Elinks
0,1,3,2,0,F,155.139000,13.473600,151.87800,148,5
1,1,2,1,0,d,109.195000,13.963700,107.49500,108,3
2,1,4,3,0,F,83.525700,5.649590,96.17450,93,3
3,1,1,3,0,F,155.107000,13.531100,151.89900,149,5
4,1,1,2,0,d,109.327000,13.894200,107.53300,108,3
5,1,1,4,0,F,83.346500,5.608940,96.00430,93,3
6,1,3,0,0,F,121.645000,9.351790,125.36500,122,4
7,1,2,0,0,g,178.111000,20.656100,171.29500,172,5
8,1,4,0,0,F,65.942400,4.211920,83.49390,85,3
9,1,3,3,0,F,121.656000,9.366370,125.37700,122,4


In [83]:
del contents[769]

In [82]:
json_file_path = "FeMappingV9.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [28]:
count=0
for i in range(len(contents)):
    if contents[i]["lpGBTs"]== [769]:
        break
    else:
        count+=1
count

693

In [ ]:
'''''''

In [85]:
rate=rates.loc[(rates["Layer"]==int(contents[769]["layer"]))& (rates["Type"]==int(contents[769]["type"]))&
                    (rates["U"]==int(contents[769]["modules"][k]["u"]))
                     &(rates["V"]==int(contents[769]["modules"][k]["v"]))]

0

In [49]:
contents.pop(769)

{'layer': 31,
 'MB': 15,
 'type': 1,
 'lpGBTs': [845],
 'modules': [{'u': 5, 'v': 3, 'elinks': [1], 'shape': 'F'},
  {'u': 6, 'v': 4, 'elinks': [1], 'shape': 'F'},
  {'u': 7, 'v': 5, 'elinks': [1], 'shape': 'F'},
  {'u': 8, 'v': 6, 'elinks': [1], 'shape': 'F'},
  {'u': 9, 'v': 7, 'elinks': [1], 'shape': 'F'},
  {'u': 10, 'v': 8, 'elinks': [1], 'shape': 'F'}]}

In [41]:
contents_original=contents

In [42]:
contents=contents2

In [50]:
contents[769]

{'layer': 31,
 'MB': 16,
 'type': 0,
 'lpGBTs': [846],
 'modules': [{'u': 3, 'v': 1, 'elinks': [2], 'shape': 'F'},
  {'u': 4, 'v': 1, 'elinks': [1], 'shape': 'F'},
  {'u': 4, 'v': 2, 'elinks': [1], 'shape': 'F'}]}

In [23]:
count

769

In [25]:
contents[769]

{'layer': 31,
 'MB': 15,
 'type': 1,
 'lpGBTs': [845],
 'modules': [{'u': 5, 'v': 3, 'elinks': [1], 'shape': 'F'},
  {'u': 6, 'v': 4, 'elinks': [1], 'shape': 'F'},
  {'u': 7, 'v': 5, 'elinks': [1], 'shape': 'F'},
  {'u': 8, 'v': 6, 'elinks': [1], 'shape': 'F'},
  {'u': 9, 'v': 7, 'elinks': [1], 'shape': 'F'},
  {'u': 10, 'v': 8, 'elinks': [1], 'shape': 'F'}]}

In [32]:
rates.loc[(rates["Layer"]==31)&(rates["Type"]==1)&(rates["U"]==5)&(rates["V"]==3)]

,Layer,U,V,Type,Shape,ADCHits,TOXHits,EvSize,EvSizeXCheck,Elinks
